<div style="text-align: right">Paul Novaes<br>July 2018</div> 

# PRIMES in NP

This notebook shows that primes have efficient short proofs of primality.

More precisely, for any prime $p$, there is a certificate of primality whose length is polynomial in the number of digits of $p$, and that can be verified in polynomial time.

Note that we are not saying that it is easy to find such certificates. We are just saying that they exist.

## PRIMES is in coNP

A related problem is to show that a number is __not__ prime. This is a much easier problem: to show that $n$ is composite it is enough to give 2 integers $a$ and $b$, greater than 1, such that 

$$n = ab$$

Here the certificate of compositeness is the pair $(a, b)$ and it can be verified by computing $ab$ and checking that it gives $n$, indeed. For example:

In [1]:
# These Mersenne numbers are known to be prime.
mersenne_521 = 2**521 - 1
mersenne_607 = 2**607 - 1

n = mersenne_521 * mersenne_607
a = mersenne_521
b = mersenne_607

print('(', a, ',', b, ')')
print('\nis a certificate of compositeness for\n')
print(n)

( 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151 , 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127 )

is a certificate of compositeness for

3646154850295011369707131011438711095400799139943170490872585628683549034362552065955809589514611470241298944167703929337528884908857116141935206466329731087514964112054543019336536216107629523597606330154669196064144182472739556974502462402438903115845725630946428943768540714098264727068026730424033578827886916761701429264950573899186177


The goal of this notebook is to show that there are also short proofs (though not as short) for the primality of primes such as the 2 Mersenne numbers above.

## Traditional primality algorithm
The traditional way of checking that a number $n$ is prime is by making sure it cannot be evenly divided by smaller numbers. Actually, checking up to $\sqrt n$ is enough.

In [2]:
# Returns True if n is prime.
def is_prime(n):
    if n <= 2: return False
    if n == 2: return True
    if n % 2 == 0: return False
    i = 3
    while (i*i <= n):
        if n % i == 0:
            return False
        i += 2
    return True

print('List of prime numbers:\n')
for n in range(1, 1000):
    if is_prime(n):
        print(n, end=' ')
print('...')

List of prime numbers:

3 5 7 11 13 17 19 23 29 31 37 41 43 47 53 59 61 67 71 73 79 83 89 97 101 103 107 109 113 127 131 137 139 149 151 157 163 167 173 179 181 191 193 197 199 211 223 227 229 233 239 241 251 257 263 269 271 277 281 283 293 307 311 313 317 331 337 347 349 353 359 367 373 379 383 389 397 401 409 419 421 431 433 439 443 449 457 461 463 467 479 487 491 499 503 509 521 523 541 547 557 563 569 571 577 587 593 599 601 607 613 617 619 631 641 643 647 653 659 661 673 677 683 691 701 709 719 727 733 739 743 751 757 761 769 773 787 797 809 811 821 823 827 829 839 853 857 859 863 877 881 883 887 907 911 919 929 937 941 947 953 967 971 977 983 991 997 ...


This works well for relatively small numbers but doesn't scale up to, say, 100 digits. Note that the time complexity is $\Theta(\sqrt n)$, that is exponential __on the number of digits__.

## Fermat's little theorem

__Fermat's little theorem__

If $p$ is prime and $0 < a < p$ then

$$a^{p-1} = 1 \mod p$$

__Proof__

For any $a$

$$(a + 1)^{p} = {p\choose 0}a^p + {p\choose 1}a^{p-1} + {p\choose 2}a^{p-2} + \cdots + {p\choose p}a^0$$

Since $p$ is prime, most of the coefficients are divisible by $p$. Indeed for $1 \leq q \leq p-1$, ${p\choose q} = {{p!} \over {q!(p-q)!}}$ has $p$ in the numerator, but not in the denominator. Therefore we have

$$(a + 1)^{p} = a^p + 1 \mod p$$

For $a = 1$, we get $$2^p = 2 \mod p$$
and using induction on $a$, we get more generally $$a^p = a \mod p$$

If $(a, p) = 1$, $a$ has an inverse (mod p), as can be seen using Euclid's algorithm, and therefore, for $0 < a < p$

$$a^{p-1} = 1 \mod p$$

Let's check the theorem for a few values:

In [3]:
# Computes a**n mod m.
def pow_mod(a, n, m):
    if a >= m:
        a %= m
    if n == 0:
        return 1
    result = pow_mod(a, n // 2, m) ** 2
    if n % 2 == 1: 
        result = a * result
    result = result % m
    return result

def fermat_test(n, a):
    return pow_mod(a, n-1, n) == 1

def check_fermat(a, n):
    assert(fermat_test(n, a))
    print(a, '** (', n-1, '- 1 ) = 1 mod', n)
    
check_fermat(2, 3)
check_fermat(2, 97)
check_fermat(23, 59)
check_fermat(2, mersenne_521)
check_fermat(2, mersenne_607)

2 ** ( 2 - 1 ) = 1 mod 3
2 ** ( 96 - 1 ) = 1 mod 97
23 ** ( 58 - 1 ) = 1 mod 59
2 ** ( 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057150 - 1 ) = 1 mod 6864797660130609714981900799081393217269435300143305409394463459185543183397656052122559640661454554977296311391480858037121987999716643812574028291115057151
2 ** ( 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728126 - 1 ) = 1 mod 531137992816767098689588206552468627329593117727031923199444138200403559860852242739162502265229285668889329486246501015346579337652707239409519978766587351943831270835393219031728127


On the other hand:

In [4]:
composite = mersenne_521 * mersenne_607
print('n =', composite)
print('\nis composite. Indeed: 2^(n-1) mod n =\n')
print(pow_mod(2, composite-1, composite))

n = 3646154850295011369707131011438711095400799139943170490872585628683549034362552065955809589514611470241298944167703929337528884908857116141935206466329731087514964112054543019336536216107629523597606330154669196064144182472739556974502462402438903115845725630946428943768540714098264727068026730424033578827886916761701429264950573899186177

is composite. Indeed: 2^(n-1) mod n =

117617898447960240594738066243890558744589682390176529192112418734563680630878699069868895603571073655147079382052381245794746597893046791796022644053827181234957536466206089534971894653069934981964122654984031075848787023021459140241103234472410417849613699642368118324583536948112226181224721873646572082806254147448685639776817059429343


Note that Fermat's test is very fast even for very large numbers, and gives us, at least in the case above, a very fast proof of compositeness. 

## Pseudoprimes

Fermat's little theorem is remarkable not only in itself, but also because its converse is almost true. But not quite as we are going to see.

__Poulet numbers__

Does $$2^{n-1} = 1 \mod n$$ imply that $n$ is prime?

Unfortunately, this is not true in general. Such numbers, composite but verifying the identity above, are called Poulet numbers.

In [5]:
def is_poulet(n):
    return fermat_test(n, 2) and not is_prime(n)

def pseudoprimes(start, end, is_pseudoprime):
    num_primes = 0
    pseudoprimes = []
    for n in range(start, end + 1):
        if is_prime(n):
            num_primes += 1
        elif is_pseudoprime(n):
            pseudoprimes.append(n)
    return pseudoprimes, num_primes

def study_pseudoprimes(start, end, is_pseudoprime, name):
    pseudoprimes_list, num_primes = pseudoprimes(start, end, is_pseudoprime)
    print('Interval[', start, ',', end, ']')
    print('  ', num_primes, 'primes')
    print('  ',  len(pseudoprimes_list), name, ':', pseudoprimes_list)

print('Poulet numbers\n')            
study_pseudoprimes(2, 1000, is_poulet, 'poulet numbers')
print()
study_pseudoprimes(1000000, 1010000, is_poulet, 'poulet numbers')

Poulet numbers

Interval[ 2 , 1000 ]
   167 primes
   3 poulet numbers : [341, 561, 645]

Interval[ 1000000 , 1010000 ]
   753 primes
   1 poulet numbers : [1004653]


While it looks like Poulet numbers are rare and this could be a good heuristic (and it seems to get better for larger numbers) it is not a __proof__ of primality.

__2-3 Pseudoprimes__

Our next attempt is to extend the test, and use 2 and 3 (and not just 2). 

Does $$2^{n-1} = 1 \mod n$$ and $$3^{n-1} = 1 \mod n$$ imply that $n$ is prime?

Things get a little better but, unfortunately, the answer is no again.

In [6]:
def is_2_3_pseudoprime(n):
    return fermat_test(n, 2) and fermat_test(n, 3) and not is_prime(n)

print('2-3 pseudoprimes\n')            
study_pseudoprimes(2, 1000, is_2_3_pseudoprime, '2-3 pseudoprimes')
study_pseudoprimes(1000000, 1010000, is_2_3_pseudoprime, '2-3 pseudoprimes')
study_pseudoprimes(1000000, 1100000, is_2_3_pseudoprime, '2-3 pseudoprimes')

2-3 pseudoprimes

Interval[ 2 , 1000 ]
   167 primes
   0 2-3 pseudoprimes : []
Interval[ 1000000 , 1010000 ]
   753 primes
   0 2-3 pseudoprimes : []
Interval[ 1000000 , 1100000 ]
   7216 primes
   4 2-3 pseudoprimes : [1024651, 1033669, 1050985, 1082809]


We could try to refine this approach, but it seems it can produce better and better heuristics, but not a proof.

## Primality witnesses

As we saw earlier, $341$ is a pseudoprime. The question is whether there is some way of distinguishing of real primes.

The answer is yes. Real primes have generators. More precisely:

if $p$ is prime, there is $g$ such that all the elements of the sequence $g^1, g^2, \ldots, g^{p-1}$ are distinct.

In [7]:
def sequence(a, n):
    res = a
    len = 1
    count = 0
    while True:
        print(res, end=' ')
        res = (res * a) % n
        len += 1
        if res == 1:
            print(res)
            break
        if (count == 30):
            break
        count += 1
    return len

def study_sequence(a, n):
    len = sequence(a, n)
    print('(', a, ',', n, ') period =', len, end='\n')

    
#study_sequence(11, 341)
    
#for i in range(2, 341):
#    study_sequence(i, 341)
    
#study_sequence(3, 341)
#study_sequence(2, 347)
#study_sequence(3, 347)

In [8]:
def is_generator(a, n):
    res = 1
    for i in range(n - 2):
        res = (res * a) % n
        if res == 1:
            return False
    return True

def gcd(a, b):
    if b == 0: return a
    return gcd(b, a % b)
        
def has_generator(n):
    for a in range(2, n):
        if gcd(a, n) == 1 and is_generator(a, n):
            return True
    return False

for n in range(3, 1000):
    prime = is_prime(n)
    generator = has_generator(n)
    assert(not (prime and not generator))
    assert(not (not prime and generator))

## Primality Verifier

We present now a function that allows to verify primality certificates.

In [9]:
# Verifies that (factors, witnesses) is a valid primality certificate for n.
# The indentation helps readability, showing the recursion level in the output.
# The verified_set is the set of primes that have been already verified, so we 
# don't verify them again.
def verify_primality_certificate(n, factors, witnesses, indentation='', verified_set=[]):
    if n == 2:
        print(indentation,"2 is a known prime")
        return True
    if n in verified_set:
        print(indentation, n, "already verified")
        return True
    print(indentation, "verifying", n, "...")
    myfactors = factors[n]
    # Verify product.
    product = 1
    for f in myfactors:
        product *= f
    if product != n - 1:
        print(indentation, product, '!=', n - 1)
        return False
    print(indentation, "product checked for", n, '- 1')
    # Verify criteria.
    witness = witnesses[n]
    if pow_mod(witness, n-1, n) != 1:
        return False
    for f in myfactors:
        if pow_mod(witness, (n-1)//f, n) == 1:
            print('Oops', witness, n, (n-1)//f, n)
            return False
    print(indentation, "witness checked for", n)
    # Recurse.
    print(indentation, 'recursing on factors', myfactors, 'of', n, '- 1')
    for f in myfactors:
        if not verify_primality_certificate(f, factors, witnesses, indentation + '  ', verified_set):
            return False
    print(indentation, '==>', n, "is prime")
    verified_set.append(n)
    return True

__Verifying 31__

In [10]:
n = 31

# (factors, witnesses) is the certificate of primality for 31.
factors = {n:[2,3,5], 3:[2], 5:[2,2]}
witnesses = {n:3, 3:2, 5:2}

verify_primality_certificate(n, factors, witnesses, '', [])

 verifying 31 ...
 product checked for 31 - 1
 witness checked for 31
 recursing on factors [2, 3, 5] of 31 - 1
   2 is a known prime
   verifying 3 ...
   product checked for 3 - 1
   witness checked for 3
   recursing on factors [2] of 3 - 1
     2 is a known prime
   ==> 3 is prime
   verifying 5 ...
   product checked for 5 - 1
   witness checked for 5
   recursing on factors [2, 2] of 5 - 1
     2 is a known prime
     2 is a known prime
   ==> 5 is prime
 ==> 31 is prime


True

__Verifying $2^{89} - 1$__

In [11]:
mersenne_89 = 2**89 - 1

# (factors, witnesses) is the certificate of primality for mersenne_89.

# Maps n to the factors of n-1.
# The n's are the primes needed in the verification.
factors = {mersenne_89:[2,3,5,17,23,89,353,397,683,2113,2931542417],
           3:[2],
           5:[2,2],
           7:[2,3],
           11:[2,5],
           17:[2,2,2,2],
           23:[2,11],
           29:[2,2,7],
           31:[2,3,5],
           89:[2,2,2,11],
           239:[2,7,17],
           353:[2,2,2,2,2,11],
           397:[2,2,3,3,11],
           683:[2,11,31],
           1451:[2,5,5,29],
           1913:[2,2,2,239],
           2113:[2,2,2,2,2,2,3,11],
           8707:[2,3,1451],
           2931542417:[2,2,2,2,11,1913,8707]}

# Maps n to its witness for Fermat's test.
# The n's are the primes needed in the verification.
witnesses = {mersenne_89:3, 
             3:2, 
             5:2, 
             7:3,
             11:2, 
             17:3, 
             23:5, 
             29:2,
             31:3, 
             89:3, 
             239:7,
             353:3, 
             397:5, 
             683:5,
             1451:2,
             1913:3,
             2113:5, 
             8707:5,
             2931542417:3}

verify_primality_certificate(mersenne_89, factors, witnesses, '', [])

 verifying 618970019642690137449562111 ...
 product checked for 618970019642690137449562111 - 1
 witness checked for 618970019642690137449562111
 recursing on factors [2, 3, 5, 17, 23, 89, 353, 397, 683, 2113, 2931542417] of 618970019642690137449562111 - 1
   2 is a known prime
   verifying 3 ...
   product checked for 3 - 1
   witness checked for 3
   recursing on factors [2] of 3 - 1
     2 is a known prime
   ==> 3 is prime
   verifying 5 ...
   product checked for 5 - 1
   witness checked for 5
   recursing on factors [2, 2] of 5 - 1
     2 is a known prime
     2 is a known prime
   ==> 5 is prime
   verifying 17 ...
   product checked for 17 - 1
   witness checked for 17
   recursing on factors [2, 2, 2, 2] of 17 - 1
     2 is a known prime
     2 is a known prime
     2 is a known prime
     2 is a known prime
   ==> 17 is prime
   verifying 23 ...
   product checked for 23 - 1
   witness checked for 23
   recursing on factors [2, 11] of 23 - 1
     2 is a known prime
     verif

True